In [5]:

# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
import pandas as pd

In [19]:
def create_model(dropout_rate, activation):
	# create model
	model = tf.keras.Sequential([
        tf.keras.layers.Input(shape= (1331, 1)),  
        tf.keras.layers.Conv1D(filters = 16, kernel_size = 8, strides = 1, activation=None),  # 1st convolutional layer: 16 filters, size 8x1
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation(activation),  # Add ReLU activation after Batch Normalization
        
        tf.keras.layers.Conv1D(filters = 32, kernel_size = 8, strides = 1, activation=None),  # 2nd convolutional layer: 64 filters, size 20x1
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation(activation),  # Add ReLU activation after Batch Normalization
        
        # Add more layers as needed
        tf.keras.layers.Flatten(),  # Flatten layer to convert 3D output to 2D for dense layers
        tf.keras.layers.Dense(16, activation=activation),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(2, activation='softmax'),
    ])
	model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
	return model

In [26]:
def _importData(file_name, classification, columns):
    ds = pd.read_csv(file_name, low_memory=False)
    y = ds[classification]
    x = ds.drop(columns, axis = 1)
    return x, y

seed = 7
tf.random.set_seed(seed)

file_train = 'forgan.csv'
classification = 'binaryclassification'
columns = ['multiclassification',	'multiname',	'binaryclassification',	'binaryname',	'realorgan']

X, Y = _importData(file_train, classification, columns)

model = KerasClassifier(model=create_model, verbose=0)
batch_size = [10, 32, 40]
epochs = [50, 100, 120]
optimizer = ['SGD', 'Adam', 'RMSprop']
activation = ['softmax', 'tanh', 'relu']
learn_rate = [0.001, 0.01]
dropout_rate = [0.2, 0.4, 0.6]

In [27]:
param_grid = dict(batch_size=batch_size, epochs=epochs, optimizer = optimizer, model__dropout_rate=dropout_rate, model__activation=activation, optimizer__learning_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [29]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.987179 using {'batch_size': 10, 'epochs': 50, 'model__activation': 'relu', 'model__dropout_rate': 0.2, 'optimizer': 'Adam', 'optimizer__learning_rate': 0.01}
0.975641 (0.031768) with: {'batch_size': 10, 'epochs': 50, 'model__activation': 'softmax', 'model__dropout_rate': 0.2, 'optimizer': 'SGD', 'optimizer__learning_rate': 0.001}
0.971795 (0.034449) with: {'batch_size': 10, 'epochs': 50, 'model__activation': 'softmax', 'model__dropout_rate': 0.2, 'optimizer': 'SGD', 'optimizer__learning_rate': 0.01}
0.969231 (0.027912) with: {'batch_size': 10, 'epochs': 50, 'model__activation': 'softmax', 'model__dropout_rate': 0.2, 'optimizer': 'Adam', 'optimizer__learning_rate': 0.001}
0.966667 (0.029179) with: {'batch_size': 10, 'epochs': 50, 'model__activation': 'softmax', 'model__dropout_rate': 0.2, 'optimizer': 'Adam', 'optimizer__learning_rate': 0.01}
0.980769 (0.019102) with: {'batch_size': 10, 'epochs': 50, 'model__activation': 'softmax', 'model__dropout_rate': 0.2, 'optimizer': 'RMSpr

In [42]:
for mean, stdev, param in zip(means, stds, params):
   if param.get('model__activation') == 'tanh' and param.get('model__dropout_rate') == 0.4 and param.get('batch_size') == 10 and param.get('optimizer') == 'Adam':   # Check if epochs is 50
        print("%f (%f) with: %r" % (mean, stdev, param))

0.915385 (0.013689) with: {'batch_size': 10, 'epochs': 50, 'model__activation': 'tanh', 'model__dropout_rate': 0.4, 'optimizer': 'Adam', 'optimizer__learning_rate': 0.001}
0.902564 (0.016115) with: {'batch_size': 10, 'epochs': 50, 'model__activation': 'tanh', 'model__dropout_rate': 0.4, 'optimizer': 'Adam', 'optimizer__learning_rate': 0.01}
0.911538 (0.025705) with: {'batch_size': 10, 'epochs': 100, 'model__activation': 'tanh', 'model__dropout_rate': 0.4, 'optimizer': 'Adam', 'optimizer__learning_rate': 0.001}
0.924359 (0.023148) with: {'batch_size': 10, 'epochs': 100, 'model__activation': 'tanh', 'model__dropout_rate': 0.4, 'optimizer': 'Adam', 'optimizer__learning_rate': 0.01}
0.955128 (0.018401) with: {'batch_size': 10, 'epochs': 120, 'model__activation': 'tanh', 'model__dropout_rate': 0.4, 'optimizer': 'Adam', 'optimizer__learning_rate': 0.001}
0.891026 (0.033579) with: {'batch_size': 10, 'epochs': 120, 'model__activation': 'tanh', 'model__dropout_rate': 0.4, 'optimizer': 'Adam', '